In [1]:
import jaydebeapi
import numpy
import pandas

<h3>環境設置</h3>

In [2]:
stock_ID="NQF"
data_table="stockInfo"

In [4]:
field_names=[
    'StockID', 'StockDate', 
    'StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
    'StockAdjClose', 'StockVol'
    ]

<h3>開啟資料庫</h3>

In [5]:
dbConnection = jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:C:/Users/user/anaconda3/Projects/data/H2",
    ["SA", ""],
    "C:/Java/h2/bin/h2-1.4.200.jar")

dbCursor = dbConnection.cursor()
#作業空間

org.h2.jdbc.JdbcSQLNonTransientConnectionException: org.h2.jdbc.JdbcSQLNonTransientConnectionException: Database may be already in use: null. Possible solutions: close all other connection(s); use the server mode [90020-200]

<h3>資料統計-按年月小計</h3>

<h3>最大,最小值,平均數</h3>

In [5]:
#用SQL做
_sql=[
    "SELECT LEFT(StockDate,7) AS YM,", #字串函數(LEFT),前七位
    "MIN(StockOpen) AS MinstockOpen,AVG(StockOpen) AS AvgstockOpen,MAX(StockOpen) AS MaxstockOpen",
    "FROM STOCKINFO",
    "WHERE StockID= '%s'" % stock_ID,
    "GROUP BY LEFT(StockDate,7)" #小計
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall() #tuple資料

rows={
    'YM':[],
    'MIN_OPEN':[],
    'AVG_OPEN':[],
    'MAX_OPEN':[]    
}
#資料結構(陣列),初始化

for row in resultSet:
    rows['YM'].append(row[0]),
    rows['MIN_OPEN'].append(row[1])
    rows['AVG_OPEN'].append(row[2])
    rows['MAX_OPEN'].append(row[3])

results=pandas.DataFrame(data=rows).set_index(['YM'] #把row指派到data裡面
results.head()

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2017-01,4876.75,5040.500000,5148.25
2017-02,5129.50,5247.973684,5352.50
2017-03,5334.25,5381.304348,5440.75
2017-04,5352.75,5442.052632,5584.50
2017-05,5575.50,5676.806818,5793.00


In [6]:
#用python做
_sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE StockID= '%s'" % stock_ID
]
#資料量較大,運算負荷較少
dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'STOCKDATE':[],
    'YM':[],
    'STOCKOPEN':[]
}
for row in resultSet:
    rows['STOCKDATE'].append(row[0])
    rows['YM'].append(row[1])
    rows['STOCKOPEN'].append(row[2])
    
results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max']) #agg計算統計量
results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]                                                                         
results.head()

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2017-01,4876.75,5040.500000,5148.25
2017-02,5129.50,5247.973684,5352.50
2017-03,5334.25,5381.304348,5440.75
2017-04,5352.75,5442.052632,5584.50
2017-05,5575.50,5676.806818,5793.00


In [7]:
#針對年份(2019,2020)
_sql=[
    "SELECT LEFT(StockDate,7) AS YM,",
    "MIN(StockOpen) AS MinstockOpen,AVG(StockOpen) AS AvgstockOpen,MAX(StockOpen) AS MaxstockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='2019' OR LEFT(STOCKDATE,4)='2020' AND StockID= '%s'" % stock_ID, #更改條件
    "GROUP BY LEFT(StockDate,7)"
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'YM':[],
    'MIN_OPEN':[],
    'AVG_OPEN':[],
    'MAX_OPEN':[]    
}

for row in resultSet:
    rows['YM'].append(row[0]),
    rows['MIN_OPEN'].append(row[1])
    rows['AVG_OPEN'].append(row[2])
    rows['MAX_OPEN'].append(row[3])

results=pandas.DataFrame(data=rows).set_index(['YM'])
results

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2019-01,6168.75,6595.333333,6844.25
2019-02,6863.75,7009.960526,7122.00
2019-03,7022.25,7259.190476,7526.75
2019-04,7430.00,7663.333333,7855.25
2019-05,7214.75,7517.511364,7848.75
2019-06,7008.75,7489.975000,7753.00
2019-07,7781.50,7915.190476,8017.50
2019-08,7349.00,7623.875000,7834.00
2019-09,7614.25,7816.912500,7916.25


In [8]:
#宣告年份變數
_year=2018
_sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='%s' AND StockID= '%s'" % (_year,stock_ID), #更改指派條件
    ]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'STOCKDATE':[],
    'YM':[],
    'STOCKOPEN':[]
}

for row in resultSet:
    rows['STOCKDATE'].append(row[0])
    rows['YM'].append(row[1])
    rows['STOCKOPEN'].append(row[2])

results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max'])
results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]                                                                                   
results.head()

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2018-01,6405.00,6774.940476,7030.00
2018-02,6328.00,6713.597222,7003.50
2018-03,6490.75,6868.773810,7136.75
2018-04,6405.75,6627.690476,6855.25
2018-05,6606.00,6868.863636,6989.50


<h3>設計函式</h3>

In [9]:
def Calc_Open(year,stock_id): #記得縮排,注意大小寫
    _sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='%s' AND StockID= '%s'" % (year,stock_id), #更改指派條件
    ]

    dbCursor.execute(' '.join(_sql))
    resultSet = dbCursor.fetchall()

    rows={
        'STOCKDATE':[],
        'YM':[],
        'STOCKOPEN':[]
    }

    for row in resultSet:
        rows['STOCKDATE'].append(row[0])
        rows['YM'].append(row[1])
        rows['STOCKOPEN'].append(row[2])

    results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max'])
    results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]           
    
    return results #return結果

In [10]:
df_2019=Calc_Open(year=2019,stock_id='NQF')
df_2019.tail()

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2019-08,7349.00,7623.875000,7834.00
2019-09,7614.25,7816.912500,7916.25
2019-10,7551.50,7856.641304,8128.50
2019-11,8094.25,8266.381579,8402.75
2019-12,8258.75,8502.434211,8801.00


In [11]:
df_all=[]
for _year in [2017,2018,2019]:
    _df=Calc_Open(year=_year,stock_id='IBM')
    df_all.append(_df)
df_all

[           MIN_OPEN    AVG_OPEN    MAX_OPEN
 YM                                         
 2017-01  166.050003  170.356001  178.470001
 2017-02  174.229996  178.461053  182.000000
 2017-03  172.690002  176.953913  181.880005
 2017-04  160.289993  167.462630  174.699997
 2017-05  150.300003  153.338638  160.050003
 2017-06  151.000000  153.910000  155.789993
 2017-07  144.380005  150.390500  155.770004
 2017-08  139.589996  142.666956  145.119995
 2017-09  142.979996  144.866499  146.679993
 2017-10  145.350006  151.659090  162.050003
 2017-11  147.729996  151.222856  154.179993
 2017-12  151.820007  154.308000  157.300003,
 Empty DataFrame
 Columns: [(MIN_OPEN,), (AVG_OPEN,), (MAX_OPEN,)]
 Index: [],
 Empty DataFrame
 Columns: [(MIN_OPEN,), (AVG_OPEN,), (MAX_OPEN,)]
 Index: []]

In [12]:
dbCursor.close()
dbConnection.close()